# Check segmentation made with Ilastik

Same as the default 1A notebook, but streamlined to quickly check settings

- Loads registered data and probability outputs from Ilastik to visualize
- Thresholds probability outputs to produce binary cell masks for the three strains

In [1]:
#next two lines make sure that Matplotlib plots are shown properly in Jupyter Notebook
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#next line is required for Napari
%gui qt

#main data analysis packages
import numpy as np
import seaborn as sns
import pandas as pd

#image viewer
import napari
from napari.utils.notebook_display import nbscreenshot

#out of memory computation
import dask.array as da
import dask_image.ndmorph as damorph

#path handling
import pathlib

#image processing packages
import skimage.filters as filters
from skimage.measure import label, regionprops_table
from skimage import morphology

#custom code for colony processing
import process_colonies as pc

#file handling
import h5py

#dask cash
from dask.cache import Cache
cache = Cache(2e9)  # Leverage two gigabytes of memory
cache.register()    # Turn cache on globally

## Set Paths and settings

In [2]:
#set path to registered file
path_regestired_im = pathlib.Path("/Volumes/ScientificData/Users/Giulia(botgiu00)/Collaborations/Ashley/2023-04-11-agar-pad-processed/Registration_max_frames/")

#set path to Ilastik output file
path_segmented_im = pathlib.Path("/Volumes/ScientificData/Users/Giulia(botgiu00)/Collaborations/Ashley/2023-04-11-agar-pad-processed/Probabilities_iteger_8bit_export/")

#set filenames
exp_name = "20230411"
pos_idx = 0

#specify properties to extract 
prop_list = ['label', 
            'area', 'centroid', 
            'axis_major_length', 'axis_minor_length']


#specify processing settings
settings = {
    #specify the order of the strains in the Ilastik layers
    'idx_SA1'   : 0, #SA1 is GFP
    'idx_SA2'   : 1, #SA2 is RFP
    'idx_BG'    : 2,
    'idx_PA'    : 3,
    #specify the segementation processing parameters for pseudomonas
    'sigma'             : 1, # sigma for gaussian filter
    'threshold_PA'      : 0.5, # threshold for segmentation
    'closing_radius_PA' : 5, # radius for closing operation
    'min_cell_area_PA'  : 20, # minimum area for a cell to be considered
    'max_hole_area_PA'  : 100, # maximum area for a hole to be filled
    #specify the segementation processing parameters for staph
    'sigma'             : 1, # sigma for gaussian filter
    'threshold_SA'      : 0.5, # threshold for segmentation
    'closing_radius_SA' : 5, # radius for closing operation
    'min_cell_area_SA'  : 20, # minimum area for a cell to be considered
    'max_hole_area_SA'  : 100, # maximum area for a hole to be filled    
}


## Load single position to check output 

In [5]:
#create metadata path and file names
metadata_path = pathlib.Path(f'./agarpad_{exp_name}.csv')
file_name_im = f"{exp_name}_reg_p{pos_idx:03d}.h5"
file_name_seg = f"{exp_name}_reg_p{pos_idx:03d}-images_Probabilities.h5"

#load metadata
df = pd.read_csv(metadata_path, index_col=0)
        
#load registered images
reg_im_file = h5py.File(path_regestired_im/file_name_im, 'r') #open 
chunk_size = (1, *reg_im_file['images'].shape[-3:])
reg_im = da.from_array(reg_im_file['images'], chunks=chunk_size)

#load segmented images
seg_im_file = h5py.File(path_segmented_im/file_name_seg, 'r') #open 
chunk_size = (1, 1,*reg_im_file['images'].shape[-2:])

seg_prob = da.from_array(seg_im_file['exported_data'], chunks=chunk_size)

#crop to max frame
max_frm = int(df.loc[f"pos{pos_idx:03d}","max_frame"]) if f"pos{pos_idx:03d}" in df.index else reg_im.shape[0]
reg_im = reg_im[:max_frm]       
seg_prob = seg_prob[:max_frm]       

condition = df.loc[f"pos{pos_idx:03d}","condition"] if f"pos{pos_idx:03d}" in df.index else ''

#convert to float if necessary
if seg_prob.dtype == 'uint8':
    seg_prob = seg_prob.astype('float32')/255

## Check raw data and Ilastik output with Napari

In [6]:
viewer = napari.view_image(reg_im,
            channel_axis=1,
            name=["red", "green", "phase"],
            colormap=["red", "green", "gray"])


#add probability layer to Napari Viewer
prop_layer1 = viewer.add_image(seg_prob[:,settings['idx_SA1'],:,:], 
                               name='pSA1',colormap='gray')
prop_layer2 = viewer.add_image(seg_prob[:,settings['idx_SA2'],:,:], 
                               name='pSA2',colormap='gray')
prop_layer3 = viewer.add_image(seg_prob[:,settings['idx_PA'],:,:], 
                               name='pPA',colormap='gray')

napari.run()

---
## Process Segmentation and check with Napari

We process segmentation and evaluate output.

We can inspect:
- `probability`: output of Ilastik
- `mask`: result after shareholding
- `mask_cleaned`: result after processing mask (filling holes and closing operation)
- `labels`: result after semantic segmentation

In [7]:
#convert to labels
SA1_labels, SA1_mask, SA1_maks_cl = pc.process_seg(seg_prob[:,settings['idx_SA1'],:,:], 
                            sigma = settings['sigma'],
                            threshold = settings['threshold_SA'],
                            closing_radius = settings['closing_radius_SA'],
                            min_cell_area = settings['min_cell_area_SA'],
                            max_hole_area = settings['max_hole_area_SA'],
                            output_intermediate=True)
                            
SA2_labels, SA2_mask, SA2_mask_cl = pc.process_seg(seg_prob[:,settings['idx_SA2'],:,:], 
                        sigma = settings['sigma'],
                        threshold = settings['threshold_SA'],
                        closing_radius = settings['closing_radius_SA'],
                        min_cell_area = settings['min_cell_area_SA'],
                        max_hole_area = settings['max_hole_area_SA'],
                        output_intermediate=True)                       

PA_labels, PA_mask, PA_mask_cl = pc.process_seg(seg_prob[:,settings['idx_PA'],:,:], 
                        sigma = settings['sigma'],
                        threshold = settings['threshold_PA'],
                        closing_radius = settings['closing_radius_PA'],
                        min_cell_area = settings['min_cell_area_PA'],
                        max_hole_area = settings['max_hole_area_PA'],
                        output_intermediate=True) 

#add to viewer:


prop_layer1.visible = False
prop_layer2.visible = False
prop_layer3.visible = False


mask_layer_SA1 = viewer.add_image(SA1_mask, name='SA1 mask',
                                  colormap='green',opacity=0.3, visible=False)
mask_layer_SA2 = viewer.add_image(SA2_mask, name='SA2 mask',
                                  colormap='red',opacity=0.3, visible=False)
mask_layer_PA = viewer.add_image(PA_mask, name='PA mask',
                                 colormap='gray',opacity=0.3, visible=False)

mask_cl_layer_SA1 = viewer.add_image(SA1_mask, name='SA1 mask cleaned',
                                     colormap='green',opacity=0.3)
mask_cl_layer_SA2 = viewer.add_image(SA2_mask, name='SA2 mask cleaned',
                                     colormap='red',opacity=0.3)
mask_cl_layer_PA = viewer.add_image(PA_mask, name='PA mask cleaned',
                                    colormap='gray',opacity=0.3)


nap_marker_labels1 = viewer.add_labels(SA1_labels, name='SA1 Labels', visible=False)
nap_marker_labels2 = viewer.add_labels(SA2_labels, name='SA2 Labels', visible=False)
nap_marker_labels3 = viewer.add_labels(PA_labels, name='PA Labels', visible=False)  

----
## Track colonies and inspect with Napari

In [8]:
#track colonies and extract properties
df_SA1 = pc.track_extract_prop(SA1_labels, prop_list, 
                            metadata = {'pos': pos_idx, 'strain':'SA1','condition':condition})

df_SA2 = pc.track_extract_prop(SA2_labels, prop_list, 
                            metadata = {'pos': pos_idx, 'strain':'SA2','condition':condition})

df_PA = pc.track_extract_prop(PA_labels, prop_list, 
                            metadata = {'pos': pos_idx, 'strain':'PA','condition':condition})

#combine dataframes
df = pd.concat([df_SA1, df_SA2, df_PA]).reset_index(drop=True)

#add tracks to viewer

nap_marker_labels1.visible = True
nap_marker_labels2.visible = True
nap_marker_labels3.visible = True

mask_layer_SA1.visible = False
mask_layer_SA2.visible = False
mask_layer_PA.visible = False

mask_cl_layer_SA1.visible = False
mask_cl_layer_SA2.visible = False
mask_cl_layer_PA.visible = False

viewer.add_tracks(pc.df_track_to_lin(df_SA1), name='Tracks SA1')
viewer.add_tracks(pc.df_track_to_lin(df_SA2), name='Tracks SA2')
viewer.add_tracks(pc.df_track_to_lin(df_PA), name='Tracks PA')
viewer.show()